In [ ]:
# from utils.segmentation import YOLOSegmentation
import cv2
import numpy as np
import rospy
from cv_bridge import CvBridge, CvBridgeError
from sensor_msgs.msg import Image
from contact_grasp.srv import segmentationSrv, segmentationSrvResponse
from utils.camera_utils import RealCamera, RealCameraROS

# Demo of segmentator 

please launch the YOLO ros segmentation with either:

roslaunch yolov8_ros segmentation_rqt.launch\
roslaunch yolov8_ros segmentation.launch

In [ ]:
print("Start")
rospy.init_node("python_node",anonymous=True)

camera_connexion = "pyWrapper"
if camera_connexion == "ROS":
    camera = RealCameraROS()
    intrinsic, distortion = camera.getIntrinsic()
elif camera_connexion == "pyWrapper":
    camera = RealCamera()
    camera.start()
    #retrieve image and depth
    for i in range(15):
        rgb, depth_image, depth_scale = camera.get_rgb_depth()

    rgb, depth_image, depth_scale = camera.get_rgb_depth()
    intrinsic, distortion = camera.getIntrinsic()
else:
    raise Exception("Please choose a valid camera connexion method: ROS or pyWrapper")

bridge = CvBridge()

while not rospy.is_shutdown():
    # img_data = rospy.wait_for_message("/camera/color/image_raw", Image, timeout=0.5)
    rgb = camera.get_rgb_only()
    bgr = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)

    #load cv image
    # img = cv2.imread("/home/vdrame/bgr.png")

    #convert to ros image
    ros_img = bridge.cv2_to_imgmsg(bgr, encoding="bgr8")

    #call service
    rospy.wait_for_service("segmentation")

    try:
        segmentation = rospy.ServiceProxy("segmentation", segmentationSrv)
        resp = segmentation(ros_img)
    except rospy.ServiceException as e:
        print(e)

    #convert to cv image
    cv_img = bridge.imgmsg_to_cv2(resp.image, desired_encoding="passthrough")
    print(np.unique(cv_img))
    #show image
    cv2.imshow("segmented image", cv_img)
    cv2.imshow("rgb", rgb)
    key=cv2.waitKey(25)
    if key == ord('q'):
        cv2.destroyAllWindows()
        break
